In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
Web_data_1 = pd.read_csv("/Users/levinschily/Desktop/IRONHACK/Week5/project/df_final_web_data_pt_1.txt")
Web_data_2 = pd.read_csv('/Users/levinschily/Desktop/IRONHACK/Week5/project/df_final_web_data_pt_2.txt')
Final_demo = pd.read_csv('/Users/levinschily/Desktop/IRONHACK/Week5/project/df_final_demo.txt')
Clients = pd.read_csv('/Users/levinschily/Desktop/IRONHACK/Week5/project/df_final_experiment_clients.txt')

combined_df = pd.concat([Web_data_1, Web_data_2], ignore_index=True)
combined_df.drop_duplicates()

merged = combined_df.merge(Clients, on='client_id', how='inner')

df = merged.merge(Final_demo, on='client_id', how='inner')
df.drop_duplicates()
df

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449826,9895983,473024645_56027518531,498981662_93503779869_272484,step_3,2017-06-15 19:52:09,NaN,10.0,121.0,49.0,M,2.0,97748.22,2.0,2.0
449827,9895983,473024645_56027518531,498981662_93503779869_272484,step_2,2017-06-15 19:50:37,NaN,10.0,121.0,49.0,M,2.0,97748.22,2.0,2.0
449828,9895983,473024645_56027518531,498981662_93503779869_272484,step_1,2017-06-15 19:50:05,NaN,10.0,121.0,49.0,M,2.0,97748.22,2.0,2.0
449829,9895983,473024645_56027518531,498981662_93503779869_272484,start,2017-06-15 19:50:00,NaN,10.0,121.0,49.0,M,2.0,97748.22,2.0,2.0


In [33]:
df["date_time"] = pd.to_datetime(df["date_time"])

df = df.sort_values(by=["Variation", "client_id", "visit_id", "process_step", "date_time"])

df_last_step = df.groupby(["Variation", "client_id", "visit_id", "process_step"]).tail(1)

df_pivot = df_last_step.pivot(index=["Variation", "client_id", "visit_id"], columns="process_step", values="date_time").reset_index()

df_pivot["start_to_step1"] = (df_pivot.get("step_1") - df_pivot.get("start")).dt.total_seconds()
df_pivot["step1_to_step2"] = (df_pivot.get("step_2") - df_pivot.get("step_1")).dt.total_seconds()
df_pivot["step2_to_step3"] = (df_pivot.get("step_3") - df_pivot.get("step_2")).dt.total_seconds()
df_pivot["step3_to_confirm"] = (df_pivot.get("confirm") - df_pivot.get("step_3")).dt.total_seconds()

avg_times = df_pivot.groupby("Variation")[["start_to_step1", "step1_to_step2", "step2_to_step3", "step3_to_confirm"]].mean().round(2)

print("\n Average time (in seconds) per step by group:")
print(avg_times)


 Average time (in seconds) per step by group:
process_step  start_to_step1  step1_to_step2  step2_to_step3  step3_to_confirm
Variation                                                                     
Control                18.49           34.41           94.62            135.73
Test                    3.53           37.50           92.83            143.38


In [35]:
import pandas as pd

# Load and clean
df['date_time'] = pd.to_datetime(df['date_time'])

df = df.sort_values(by=["Variation", "client_id", "visit_id", "process_step", "date_time"])

df_last = df.groupby(["Variation", "client_id", "visit_id", "process_step"]).tail(1)

df_pivot = df_last.pivot(index=["Variation", "client_id", "visit_id"], 
                         columns="process_step", values="date_time").reset_index()

df_pivot = df_pivot[
    (df_pivot["start"] <= df_pivot["step_1"]) &
    (df_pivot["step_1"] <= df_pivot["step_2"]) &
    (df_pivot["step_2"] <= df_pivot["step_3"]) &
    (df_pivot["step_3"] <= df_pivot["confirm"])
]

df_pivot["start_to_step1"] = (df_pivot["step_1"] - df_pivot["start"]).dt.total_seconds()
df_pivot["step1_to_step2"] = (df_pivot["step_2"] - df_pivot["step_1"]).dt.total_seconds()
df_pivot["step2_to_step3"] = (df_pivot["step_3"] - df_pivot["step_2"]).dt.total_seconds()
df_pivot["step3_to_confirm"] = (df_pivot["confirm"] - df_pivot["step_3"]).dt.total_seconds()

avg_times = df_pivot.groupby("Variation")[[
    "start_to_step1", "step1_to_step2", "step2_to_step3", "step3_to_confirm"
]].mean().round(2)

print("\n Average time (in seconds) per step by group:")
print(avg_times)


📊 Average time (in seconds) per step by group:
process_step  start_to_step1  step1_to_step2  step2_to_step3  step3_to_confirm
Variation                                                                     
Control                50.60           49.58           94.69            135.51
Test                   43.55           42.46           93.24            143.01
